In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime,nltk, warnings
import matplotlib.cm as cm
import itertools

from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_score
from sklearn import preprocessing,model_selection,metrics,feature_selection
from sklearn.model_selection import GridSearchCV,learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors,linear_model,svm,tree,ensemble
from wordcloud import WordCloud,STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display,HTML


In [2]:
df_initial = pd.read_csv('./data/xxx.csv',encoding="ISO-8859-1",
                         dtype={'CustomerID': str,'InvoiceID': str})

print('Dataframe dimensions:', df_initial.shape)
df_initial['InvoiceDate']=pd.to_datetime(df_initial['InvoiceDate'])


('Dataframe dimensions:', (53429, 8))


In [3]:
display(df_initial[:5])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [4]:
tab_info=pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values(nb)'}))

tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.rename(index={0:'null values (%)'}))

print display(tab_info)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,float64,datetime64[ns],float64,object,object
null values(nb),0,0,158,1,1,1,19623,1
null values (%),0,0,0.29572,0.00187164,0.00187164,0.00187164,36.7272,0.00187164


None


In [5]:
df_initial.dropna(axis=0,subset=['CustomerID'],inplace=True)
print('Dataframe dimensions:',df_initial.shape)

tab_info=pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values (nb)'}))

tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.rename(index={0:'null values(%)'}))

display(tab_info)

('Dataframe dimensions:', (33806, 8))


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,float64,datetime64[ns],float64,object,object
null values (nb),0,0,0,0,0,0,0,0
null values(%),0,0,0,0,0,0,0,0


In [6]:
print('Duplicate data entries:{}'.format(df_initial.duplicated().sum()))
df_initial.drop_duplicates(inplace=True)


Duplicate data entries:539


In [7]:
temp=df_initial[['CustomerID','InvoiceNo','Country']].groupby(['CustomerID','InvoiceNo','Country']).count()

temp=temp.reset_index(drop=False)
countries=temp['Country'].value_counts()
print('No of countries in dataframe: {}'.format(len(countries)))


No of countries in dataframe: 23


In [8]:
temp_no_of_order_per_count=df_initial[['CustomerID','Country']].groupby(['Country']).count()
temp_no_of_order_per_count=temp_no_of_order_per_count.reset_index(drop=False)

print(temp_no_of_order_per_count.sort_values(by='CustomerID',ascending=False).rename(index=str,columns={"CustomerID:":"Country wise number of order"}))

            Country  CustomerID
22   United Kingdom       30039
9           Germany         841
8            France         677
6              EIRE         390
19            Spain         302
18         Portugal         176
16           Norway         147
0         Australia         102
2           Belgium          99
4            Cyprus          82
15      Netherlands          72
13            Japan          69
21      Switzerland          49
12            Italy          42
20           Sweden          41
14        Lithuania          35
10          Iceland          31
5           Denmark          20
7           Finland          17
3   Channel Islands          17
1           Austria           9
17           Poland           8
11           Israel           2


In [9]:
pd.DataFrame([{'products': len(df_initial['StockCode'].value_counts()),    
               'transactions': len(df_initial['InvoiceNo'].value_counts()),
               'customers': len(df_initial['CustomerID'].value_counts()),  
              }], columns = ['products', 'transactions', 'customers'], 
              index = ['quantity'])

,products,transactions,customers
quantity,2516,2085,1082


In [10]:
temp=df_initial.groupby(by=['CustomerID','InvoiceNo'],as_index=False)['InvoiceDate'].count()

nb_products_per_basket=temp.rename(columns={'InvoiceDate':'Number of products'})
nb_products_per_basket[:10].sort_values('CustomerID')

,CustomerID,InvoiceNo,Number of products
0,12347,537626,31
1,12348,539318,17
2,12370,538826,82
3,12370,539330,8
4,12377,539488,43
5,12383,539840,37
6,12386,537676,8
7,12386,540280,2
8,12393,540700,19
9,12395,537026,12


In [11]:
nb_products_per_basket['order_cancelled']=nb_products_per_basket['InvoiceNo'].apply(lambda x:int('C' in x)) 
display(nb_products_per_basket[:5])

n1=nb_products_per_basket['order_cancelled'].sum()
n2=nb_products_per_basket[3]

percentage=(n1/n2)*100
print('Number of orders cancelled:{}/{} ({:.2f}%) '.format(n1,n2,percentage))

,CustomerID,InvoiceNo,Number of products,order_cancelled
0,12347,537626,31,0
1,12348,539318,17,0
2,12370,538826,82,0
3,12370,539330,8,0
4,12377,539488,43,0


KeyError: 3

In [12]:
display(df_initial.sort_values('CustomerID')[:5])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14960,537626,84997D,PINK 3 PIECE POLKADOT CUTLERY SET,6.0,2010-12-07 14:57:00,3.75,12347,Iceland
14959,537626,84997B,RED 3 PIECE RETROSPOT CUTLERY SET,6.0,2010-12-07 14:57:00,3.75,12347,Iceland
14968,537626,20782,CAMOUFLAGE EAR MUFF HEADPHONES,6.0,2010-12-07 14:57:00,5.49,12347,Iceland
14966,537626,84558A,3D DOG PICTURE PLAYING CARDS,24.0,2010-12-07 14:57:00,2.95,12347,Iceland
14965,537626,21731,RED TOADSTOOL LED NIGHT LIGHT,12.0,2010-12-07 14:57:00,1.65,12347,Iceland


In [13]:
df_check=df_initial[df_initial['Quantity']<0][['CustomerID','Quantity','StockCode','Description','UnitPrice']]

for index,col in df_check.iterrows():
    if df_initial[(df_initial['CustomerID'] == col[0]) & (df_initial['Quantity']==-col[1])&(df_initial['Description']==col[2])].shape[0]==0:
        print(df_check.loc[index])
        print(15* '-'+'>'+'  HYPOTHESİS NOT FULLFILLED')
        break
              

CustomerID        14527
Quantity             -1
StockCode             D
Description    Discount
UnitPrice          27.5
Name: 141, dtype: object
--------------->  HYPOTHESİS NOT FULLFILLED


In [14]:
isdf_check=df_initial[(df_initial['Quantity']<0)&(df_initial['Description'] !='Discount')]
[['CustomerID','Quantity','StockCode','Description','UnitPrice']]


for index,col in df_check.iterrows():
    if df_initial[(df_initial['CustomerID']==col[0])& (df_initial['Quantity']==-col[1]) 
                  & (df_initial['Description'] ==col[2])].shape[0]==0:
        print(index,df_check.loc[index])
        break
        

(141, CustomerID        14527
Quantity             -1
StockCode             D
Description    Discount
UnitPrice          27.5
Name: 141, dtype: object)


In [15]:
df_cleaned = df_initial.copy(deep = True)
df_cleaned['QuantityCanceled'] = 0

entry_to_remove = [] ; doubtfull_entry = []

for index, col in  df_initial.iterrows():
    if (col['Quantity'] > 0) or col['Description'] == 'Discount': continue        
    df_test = df_initial[(df_initial['CustomerID'] == col['CustomerID']) &
                         (df_initial['StockCode']  == col['StockCode']) & 
                         (df_initial['InvoiceDate'] < col['InvoiceDate']) & 
                         (df_initial['Quantity']   > 0)].copy()

    # Cancelation WITHOUT counterpart
    if (df_test.shape[0] == 0): 
        doubtfull_entry.append(index)
   
    # Cancelation WITH a counterpart
    elif (df_test.shape[0] == 1): 
        index_order = df_test.index[0]
        df_cleaned.loc[index_order, 'QuantityCanceled'] = -col['Quantity']
        entry_to_remove.append(index)        
   
    # Various counterparts exist in orders: we delete the last one
    elif (df_test.shape[0] > 1): 
        df_test.sort_index(axis=0 ,ascending=False, inplace = True)        
        for ind, val in df_test.iterrows():
            if val['Quantity'] < -col['Quantity']: continue
            df_cleaned.loc[ind, 'QuantityCanceled'] = -col['Quantity']
            entry_to_remove.append(index) 
            break

In [16]:
print("entry_to_remove: {}".format(len(entry_to_remove)))
print("doubtfull_entry: {}".format(len(doubtfull_entry)))

entry_to_remove: 442
doubtfull_entry: 455


In [17]:
df_cleaned.drop(entry_to_remove,axis=0,inplace=True)
df_cleaned.drop(doubtfull_entry,axis=0,inplace=True)
remaining_entries=df_cleaned[(df_cleaned['Quantity']<0)&(df_cleaned['StockCode'] != 'D')]

print("nb of entries to delete: {}".format(remaining_entries.shape[0]))
remaining_entries[:5]

nb of entries to delete: 0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityCanceled


In [18]:
list_special_codes = df_cleaned[df_cleaned['StockCode'].str.contains('^[a-zA-Z]+', regex=True)]['StockCode'].unique()
list_special_codes

array([u'POST', u'D', u'C2', u'M', u'BANK CHARGES'], dtype=object)

In [19]:
df_cleaned['TotalPrice']=df_cleaned['UnitPrice']*(df_cleaned['Quantity']-df_cleaned['QuantityCanceled'])
df_cleaned.sort_values('CustomerID')[:5]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityCanceled,TotalPrice
14968,537626,20782,CAMOUFLAGE EAR MUFF HEADPHONES,6.0,2010-12-07 14:57:00,5.49,12347,Iceland,0.0,32.94
14955,537626,21171,BATHROOM METAL SIGN,12.0,2010-12-07 14:57:00,1.45,12347,Iceland,0.0,17.40
14954,537626,85167B,BLACK GRAND BAROQUE PHOTO FRAME,30.0,2010-12-07 14:57:00,1.25,12347,Iceland,0.0,37.50
14953,537626,22212,FOUR HOOK WHITE LOVEBIRDS,6.0,2010-12-07 14:57:00,2.10,12347,Iceland,0.0,12.60
14952,537626,22729,ALARM CLOCK BAKELIKE ORANGE,4.0,2010-12-07 14:57:00,3.75,12347,Iceland,0.0,15.00


In [20]:
temp=df_cleaned.groupby(by=['CustomerID','InvoiceNo'],as_index=False)['TotalPrice'].sum()
basket_price=temp.rename(columns={'TotalPrice':'Basket Price'})

df_cleaned['InvoiceDate_int']=df_cleaned['InvoiceDate'].astype('int64')
temp=df_cleaned.groupby(by=['CustomerID','InvoiceNo'],as_index=False)['InvoiceDate_int'].mean()

df_cleaned.drop('InvoiceDate_int',axis=1,inplace=True)
basket_price.loc[:,'InvoiceDate']=pd.to_datetime(temp['InvoiceDate_int'])

basket_price=basket_price[basket_price['Basket Price'] >0]
basket_price.sort_values('CustomerID')[:20]

,CustomerID,InvoiceNo,Basket Price,InvoiceDate
0,12347,537626,711.79,2010-12-07 14:57:00.000001024
1,12348,539318,892.80,2010-12-16 19:09:00.000000000
2,12370,538826,1587.07,2010-12-14 12:58:00.000002816
3,12370,539330,277.20,2010-12-17 09:38:00.000000000
4,12377,539488,1001.52,2010-12-20 09:37:00.000001280
5,12383,539840,600.72,2010-12-22 14:28:00.000000000
6,12386,537676,258.90,2010-12-08 09:53:00.000000000
7,12386,540280,143.00,2011-01-06 12:37:00.000000000
8,12393,540700,462.50,2011-01-11 09:46:59.999999744
9,12395,537026,346.10,2010-12-03 16:35:00.000000000


In [21]:
price_range=[0,50,100,200,500,1000,5000,50000]
count_price=[]

for i,price in enumerate(price_range):
    if i==0: continue
    val=basket_price[(basket_price['Basket Price']<price)&(basket_price['Basket Price']>price_range[i-1])]['Basket Price'].count()
    count_price.append(val)
    
    
plt.rc('font', weight='bold')
f, ax = plt.subplots(figsize=(11, 6))
colors = ['yellowgreen', 'gold', 'wheat', 'c', 'violet', 'royalblue','firebrick']
labels = [ '{}<.<{}'.format(price_range[i-1], s) for i,s in enumerate(price_range) if i != 0]
sizes  = count_price
explode = [0.0 if sizes[i] < 100 else 0.0 for i in range(len(sizes))]
ax.pie(sizes, explode = explode, labels=labels, colors = colors,
       autopct = lambda x:'{:1.0f}%'.format(x) if x > 1 else '',
       shadow = False, startangle=0)
ax.axis('equal')
f.text(0.5, 1.01, "Distribution of order amounts", ha='center', fontsize = 18);


In [23]:
is_noun = lambda pos: pos[:2] == 'NN'

def keywords_inventory(dataframe, colonne = 'Description'):
    stemmer = nltk.stem.SnowballStemmer("english")
    keywords_roots  = dict()  # collect the words / root
    keywords_select = dict()  # association: root <-> keyword
    category_keys   = []
    count_keywords  = dict()
    icount = 0
    for s in dataframe[colonne]:
        if pd.isnull(s): continue
        lines = s.lower()
        tokenized = nltk.word_tokenize(lines)
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
        
        for t in nouns:
            t = t.lower() ; racine = stemmer.stem(t)
            if racine in keywords_roots:                
                keywords_roots[racine].add(t)
                count_keywords[racine] += 1                
            else:
                keywords_roots[racine] = {t}
                count_keywords[racine] = 1
    
    for s in keywords_roots.keys():
        if len(keywords_roots[s]) > 1:  
            min_length = 1000
            for k in keywords_roots[s]:
                if len(k) < min_length:
                    clef = k ; min_length = len(k)            
            category_keys.append(clef)
            keywords_select[s] = clef
        else:
            category_keys.append(list(keywords_roots[s])[0])
            keywords_select[s] = list(keywords_roots[s])[0]
                   
    print("number of keywords in variable '{}': {}".format(colonne,len(category_keys)))
    return category_keys, keywords_roots, keywords_select, count_keywords


In [33]:
df_produits=pd.DataFrame(df_initial['Description'].unique()).rename(columns={0:'Description'})

In [34]:
keywords, keywords_roots, keywords_select, count_keywords = keywords_inventory(df_produits)


LookupError: 
**********************************************************************
  Resource u'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/zenon/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - u''
**********************************************************************

In [32]:
list_products = []
for k,v in count_keywords.items():
    list_products.append([keywords_select[k],v])
list_products.sort(key = lambda x:x[1], reverse = True)

NameError: name 'count_keywords' is not defined